## Introduction
In this document, all the test and runs done for the ABM can be found. 
It should be noted that a for each batchrun, it is recommended to only collect the relevant variables (the ones which are used to create the plots) in the datacollector (model3.py line 230-450), the others were set off during the iterations, otherwise my laptop stopped working.

First, a convergence test was done, and afterwards the model output is shown

## Convergence test
This is conducted to see how many runs of the ABM are required to have a reliable result. Based on this test, a number of 150 iterations were chosen.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from Model3 import  RiverDeltaModel
from mesa import batch_run
params = {"num_agents":[1000]}

# Set up batchrunner
result_batch = batch_run(
    RiverDeltaModel,
    parameters=params,
    iterations=500,
    max_steps=300,
    number_processes= 8,
    data_collection_period=10,
    display_progress=True,
)
df = pd.DataFrame(result_batch)
df = df[df["Step"]==df["Step"].max()]
df = df.reset_index()

# List of metrics to plot
metrics = ["Migrated_households"]

# Calculate running mean and standardize for each metric
for metric in metrics:
    df[f'{metric}_running_mean'] = df[metric].expanding().mean()
    mean = df[f'{metric}_running_mean'].mean()
    std = df[f'{metric}_running_mean'].std()
    df[f'{metric}_standardized'] = (df[f'{metric}_running_mean'] - mean) / std

# Plotting on 2x2 subplots
fig, axs = plt.subplots(2, 1, figsize=(12, 10))
axs = axs.flatten()

for idx, metric in enumerate(metrics):
    axs[idx].plot(df.index + 1, df[f'{metric}_standardized'], label=f'Std Running Mean: {metric}')
    # Add the 0.5 and -0.5 blue reference lines with one shared label
    axs[idx].axhline(1, color="blue", linestyle="--", linewidth=1)
    axs[idx].axhline(-1, color="blue", linestyle="--", linewidth=1, label="One standardevation threshold")
    axs[idx].set_title(f'Convergence: {metric}')
    axs[idx].set_xlabel('Model Run')
    axs[idx].set_ylabel('Standardized Running Mean')
    axs[idx].grid(True)

    # Add both plot and reference line to the legend
    axs[idx].legend()

plt.tight_layout()
plt.show()

In [ ]:
df

## MODEL OUTPUT
The model is run for 25 years and 150 iterations, to look at the model output. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from Model3 import RiverDeltaModel
from mesa import batch_run
import statsmodels.api as sm

# Use the batchrunner to run the model with 150 iterations, and collect data each step.  
params = {"num_agents": [1000]}
result_batch = batch_run(
    RiverDeltaModel,
    parameters=params,
    iterations=150,
    max_steps=300,  
    number_processes=8,
    data_collection_period=1,  # Collect data each step
    display_progress=True,
)

# Save the results in a dataframe
df = pd.DataFrame(result_batch)

#### Number of wage workers

In [ ]:
# Group by step, calculate the average and std number of wage workers
grouped = df.groupby("Step")["Number_of_wage_workers"].agg(["mean", "std"])

# Use years instead of months, start in 2014
year_index = grouped.index / 12 + 2014

# Plot
plt.figure(figsize=(6, 4))
plt.plot(year_index, grouped["mean"], label="Number of wage workers")
plt.fill_between(year_index,
                 grouped["mean"] - grouped["std"],
                 grouped["mean"] + grouped["std"],
                 alpha=0.3)
plt.xlabel("Year")
plt.ylabel("Wage workers")
plt.title("Number of wage workers over time in ABM")
plt.legend()
plt.grid(True)


plt.tight_layout()
plt.show()


#### Savings of landless households

In [ ]:
# Group by step, calculate the average and std  for the savings of landless households
grouped = df.groupby("Step")["Savings_landless_hh"].agg(["mean", "std"])

# Use years instead of months, start in 2014
year_index = grouped.index / 12 + 2014

# Plot
plt.figure(figsize=(6, 4))
plt.plot(year_index, grouped["mean"], label="Savings")
plt.fill_between(year_index,
                 grouped["mean"] - grouped["std"],
                 grouped["mean"] + grouped["std"],
                 alpha=0.3)
plt.xlabel("Year")
plt.ylabel("Savings (VND)")
plt.title("Average savings landless households in ABM")
plt.legend()

plt.grid(True)


plt.tight_layout()
plt.show()

#### Number of migrated households and household members

In [ ]:
# Group by step, calculate the average and std  for the migrated households and migrated individuals
migration_grouped = df.groupby('Step')[['Migrated_households', 'Migrated_individuals']].agg(['mean', 'std'])

# Use years instead of months, start in 2014
year_index = migration_grouped.index / 12 + 2014

hh_mean = migration_grouped[('Migrated_households', 'mean')]
hh_std = migration_grouped[('Migrated_households', 'std')]

ind_mean = migration_grouped[('Migrated_individuals', 'mean')]
ind_std = migration_grouped[('Migrated_individuals', 'std')]

# Plot
plt.figure(figsize=(6,4))

# Plot the migrated households over time
plt.plot(year_index, hh_mean, label='Migrated households', color='red')
plt.fill_between(year_index, hh_mean - hh_std, hh_mean + hh_std, alpha=0.3, color='red')

# plot the migrated individuals over time
plt.plot(year_index, ind_mean, label='Migrated individuals', color='blue')
plt.fill_between(year_index, ind_mean - ind_std, ind_mean + ind_std, alpha=0.3, color='blue')

# Dotted threshold line with the start number of households
plt.axhline(y=250, color='green', linestyle='--', linewidth=2, label='Start number of households')

# Fix axis labels, implement a grid
plt.xlabel('Years')
plt.ylabel('Migrated agents')
plt.title('Migration households and household members over time in ABM')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


#### Number of farmers over time per crop category

In [ ]:
# Strip the columns, to make sure there are no spaces left
df.columns = [col.strip() for col in df.columns]

# Mapping crops to colors, to make sure all graphs for ABM and SD can be compared
crop_colors = {
    "Rice": "red",
    "Maize": "orange",
    "Coconut": "green",
    "Shrimp": "blue"
}

# Setup the land categories and crop types, to make it easier to plot
land_categories = ["small", "medium", "large"]
crops = ["Rice", "Maize", "Coconut", "Shrimp"]

# Use years instead of months, start in 2014
years = df["Step"] / 12 + 2014 

# Create 1 figure with 3 subplots
fig, axes = plt.subplots(3, 1, figsize=(9, 12), sharex=True)

for i, land in enumerate(land_categories):
    ax = axes[i]
    
    for crop in crops:
        col = f"Count {crop} {land}"
        if col in df.columns:
            grouped = df.groupby("Step")[col].mean()
            std = df.groupby("Step")[col].std()

            year_index = grouped.index / 12 + 2014  
            ax.plot(year_index, grouped.values, label=crop, color=crop_colors[crop])
            ax.fill_between(year_index, grouped - std, grouped + std, color=crop_colors[crop], alpha=0.2)

    ax.set_title(f"Number of farms over time with {land} land in ABM")
    ax.set_ylabel("Number of farms")
    ax.grid(True)

# Set X-axes
axes[-1].set_xlabel("Year")
axes[-1].set_xlim([2014, 2014 + 25])  

# Add a legend
axes[0].legend(loc="upper left")

plt.tight_layout()
plt.show()



#### Multiple extra variables over time

In [ ]:
# Strip the columns, to make sure there are no spaces left
df.columns = [col.strip() for col in df.columns]

crop_colors = {
    "Rice": "red",
    "Maize": "orange",
    "Coconut": "green",
    "Shrimp": "blue"
}

# Select the variables required to make the plots
variables = ["Savings", "Yield", "Income", "Wage costs", "Debt ratio"]
land_categories = ["small", "medium", "large"]
crops = ["Rice", "Maize", "Coconut", "Shrimp"]

# Determine the units for the axes
units = {
    "Savings": "VND",
    "Income": "VND",
    "Yield": "kg",
    "Wage costs": "VND",
    "Debt ratio": ""  
}

# Create three subplots
for var in variables:
    fig, axes = plt.subplots(3, 1, figsize=(9,12), sharex=True)

    for i, land in enumerate(land_categories):
        ax = axes[i]

        for crop in crops:
            col = f"{var} {crop} {land}"
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce") 
                grouped = df.groupby("Step")[col].mean()
                std = df.groupby("Step")[col].std()
                years = grouped.index / 12 + 2014

                ax.plot(years, grouped, label=crop, color=crop_colors[crop])
                ax.fill_between(years, grouped - std, grouped + std, color=crop_colors[crop], alpha=0.2)

        unit_label = f" ({units[var]})" if units[var] else ""
        ax.set_title(f"{var} over time with {land} land in ABM")
        ax.set_ylabel(f"{var}{unit_label}")
        ax.grid(True)

        # Specifieke y-as limieten voor 'Savings'
        if var == "Savings":
            if land == "small":
                ax.set_ylim([-2.5e8, 9e8])
            elif land == "medium":
                ax.set_ylim([-0.75e9, 6e9])


    axes[-1].set_xlabel("Year")
    axes[-1].set_xlim([2014, 2014 + 25])
    axes[0].legend(loc="upper left")

    plt.tight_layout()
    plt.show()
